# Momentum signals

In this example, we will examine extracting and drawing momentum signals.

We will

* Download pair and exchange map ("the trading universe")

* Create a momentum function

* Draw a momentum signal on OHLCV chart


## About momentum signals

In financial technical analysis, momentum (MTM) and rate of change (ROC) are simple indicators showing the difference between today's closing price and the close N days ago. Momentum is the absolute difference in stock, commodity:

[Read more in Wikipedia](https://en.wikipedia.org/wiki/Momentum_(technical_analysis).

## Getting started

First let's import libraries and initialise our dataset client.

In [67]:
import pandas as pd

try:
    import capitalgram
except ImportError:
    !pip install -e git+https://github.com/miohtama/capitalgram-onchain-dex-quant-data.git#egg=capitalgram
    import site
    site.main()

from capitalgram.client import Capitalgram

client = Capitalgram.create_jupyter_client()

Started Capitalgram in Jupyter notebook environment, configuration is stored in /Users/mikkoohtamaa/.capitalgram


## Importing data

We will operate on 4 hour candles across all blockchains and exchanges.
We will limit the active dataset by doing a prefiltering and cull out
trading pairs that have never been too active.
This will greatly decrease computation times, because the dataset will be much smaller.
Note that prefiltering pairs like this  will introduce survivorship bias.

In [68]:
from pyarrow import Table
from capitalgram.exchange import ExchangeUniverse
from capitalgram.pair import PandasPairUniverse
from capitalgram.timebucket import TimeBucket
from capitalgram.candle import GroupedCandleUniverse

# Exchange map data is so small it does not need any decompression
exchange_universe: ExchangeUniverse = client.fetch_exchange_universe()

# We make this notebook derministic by having start and end date for all data
start = pd.Timestamp("2020-10-01")
end = pd.Timestamp("2021-08-01")

# Fetch all trading pairs across all exchanges
# We limit the test set to pairs with minimun 50 buys and 50 sells over the lifetime
pair_table: Table = client.fetch_pair_universe()
pairs = pair_table.to_pandas()
pairs = pairs.loc[
        (pairs['buy_count'] >= 50) &
        (pairs['sell_count'] >= 50)
    ]
pair_universe = PandasPairUniverse()
pair_ids = pair_universe.get_all_pair_ids()

# Fetch all candles and filter out candles that are
# belong to the pair universe
candle_table: Table = client.fetch_all_candles(TimeBucket.h4)
candles = candle_table.to_pandas()
candles = candles.loc[
    candles["pair_id"].isin(pair_ids) &
    (candles["timestamp"] >= start) &
    (candles["timestamp"] < end)
]
candle_universe = GroupedCandleUniverse(candles)

## Momentum function

We use the momentum function from [Teddy Kroker's post](https://teddykoker.com/2019/05/momentum-strategy-from-stocks-on-the-move-in-python/).

The original code uses [BUS/252, or 252 business days per year rule](https://quant.stackexchange.com/questions/32914/bus-252-accrual-why-252). Because we are trading crypto and we are trading
4 hour candles, to annualise the momentum we use a different multiplier for the slope.

In [ ]:
import numpy as np

from scipy.stats import linregress

DAYS_IN_YEAR = 365
CANDLE_HOURS = 4
ANNUALISE_MULTIPLIER = DAYS_IN_YEAR * (CANDLE_HOURS/24)


def calculate_momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    # annualize slope and multiply by R^2
    return ((1 + slope) ** ANNUALISE_MULTIPLIER) * (rvalue ** 2)

## Single pair momentum

Let's pick a pair and calculate a momentum signal for it.

We will use AAVE-ETH pair on SushiSwap.

In [69]:
from capitalgram.chain import ChainId
from capitalgram.pair import DEXPair

# Filter down to pairs that only trade on Sushiswap
sushi_swap = exchange_universe.get_by_name_and_chain(ChainId.ethereum, "sushiswap")

aave_eth: DEXPair = pair_universe.get_one_pair_from_pandas_universe(
    sushi_swap.exchange_id,
    "WETH",
    "AAVE")

Let's calculate momentum signal for this pair

In [70]:
closes = candle_universe.get_candles_by_pair(aave_eth.pair_id)["close"]
momentum_signal = calculate_momentum(closes)

,open,high,low,close
timestamp,,,,
2020-10-01,$24.1M,$28.3M,$13.2M,$14.6M
2020-10-02,$14.5M,$23.2M,$14.5M,$23.1M
2020-10-03,$23.1M,$23.4M,$22.3M,$22.7M
2020-10-04,$22.7M,$22.8M,$22.1M,$22.3M
2020-10-05,$22.3M,$22.4M,$16.9M,$17.1M
2020-10-06,$17.1M,$17.7M,$16.2M,$16.7M
2020-10-07,$16.7M,$16.7M,$15.1M,$16.3M
2020-10-08,$16.4M,$16.8M,$14.7M,$15.0M
2020-10-09,$15.0M,$19.5M,$15.0M,$19.4M


Let's see what kind of values we got by doing a table output, plot for close price, plot for momentum.